In [1]:
import cv2
import torch
import pandas as pd
from torch.utils import data
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.transforms import InterpolationMode
import pandas as pd
import pytorch_lightning as pl
from torchvision.io import read_image
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

## Load the data

In [2]:
test_path = 'data/chest_xray/test'
train_path = 'data/chest_xray/train'
val_path = 'data/chest_xray/val'

In [3]:
img_info = pd.read_csv('./img_info.csv')
test_df = img_info.loc[img_info.loc[:, 'data'] == 'Test']
train_df = img_info.loc[img_info.loc[:, 'data'] == 'Train']
val_df = test_df = img_info.loc[img_info.loc[:, 'data'] == 'Validation']

In [4]:
normal_images = train_df.loc[train_df.loc[:,'label'] == 0]
normal_images.loc[:,'imbalance'] = 1
train_df = pd.concat([train_df, normal_images, normal_images]).reset_index(drop=True)
print('Percentage of Normal Images in the Training Data: {}'.format(
    len(train_df.loc[train_df.loc[:,'label'] == 0])/len(train_df.loc[:,'label'])))
print('Percentage of Pneumonia Images in the Training Data: {}'.format(
    len(train_df.loc[train_df.loc[:,'label'] == 1])/len(train_df.loc[:,'label'])))

Percentage of Normal Images in the Training Data: 0.5093694606229425
Percentage of Pneumonia Images in the Training Data: 0.4906305393770575


/var/folders/gm/qwgd8hzn23n90l6twpb5dty40000gn/T/ipykernel_47421/907957275.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal_images.loc[:,'imbalance'] = 1


In [5]:
class CustomImageDataset(Dataset):
    def __init__(self, df, img_dir_path, transforms=None):
        """
        You can set your custom dataset to take in more parameters than specified
        here. But, I recommend at least you start with the three I listed here,
        as these are standard

        csv_file (str): file path to the csv file you created /
        df (pandas df): pandas dataframe

        img_dir_path: directory path to your images
        transform: Compose (a PyTorch Class) that strings together several
          transform functions (e.g. data augmentation steps)

        One thing to note -- you technically could implement `transform` within
        the dataset. No one is going to stop you, but you can think of the
        transformations/augmentations you do as a hyperparameter. If you treat
        it as a hyperparameter, you want to be able to experiment with different
        transformations, and therefore, it would make more sense to decide those
        transformations outside the dataset class and pass it to the dataset!
        """
        self.img_labels = df
        self.img_dir = img_dir_path
        self.transforms = transforms
        
    def __len__(self):
        """
        Returns: (int) length of your dataset
        """
        return len(self.img_labels)

    def __getitem__(self, idx):
        """
        Loads and returns your sample (the image and the label) at the
        specified index

        Parameter: idx (int): index of interest

        Returns: image, label
        """

        img_path =  self.img_labels.iloc[idx, 1]
        
        image = Image.open(img_path).convert('RGB')

        label = self.img_labels.iloc[idx, -1]
        
        imbalance = self.img_labels.iloc[idx, -2]

        if self.transforms:
            
            if imbalance and not label:
                image = transforms(image)
                image = imbalance_transform(image)
                
            else:
                image = transforms(image)
               
        return image, label

In [6]:
transforms = T.Compose(
    [
        T.Resize((224,224), antialias=None, interpolation=InterpolationMode.BICUBIC),
        T.RandomApply([
            T.GaussianBlur(kernel_size=(5,5), sigma=(0.1, 0.2))
        ], p=0.5),
        T.RandomEqualize(),
        T.ToTensor()
    ]
)


imbalance_transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomRotation(degrees=10),
    T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    T.RandomErasing(p=0.2)
])

In [7]:
training_data = CustomImageDataset(train_df, train_path, transforms=transforms)
val_data = CustomImageDataset(val_df, val_path, transforms=transforms)
test_data = CustomImageDataset(test_df, test_path, transforms=transforms)

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [8]:
training_data[0]

(tensor([[[0.0941, 0.1098, 0.1098,  ..., 0.1059, 0.1137, 0.0392],
          [0.0941, 0.1098, 0.1098,  ..., 0.1098, 0.1098, 0.0392],
          [0.0902, 0.1098, 0.1137,  ..., 0.1098, 0.1098, 0.0392],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0078, 0.0078, 0.0078],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0.0941, 0.1098, 0.1098,  ..., 0.1059, 0.1137, 0.0392],
          [0.0941, 0.1098, 0.1098,  ..., 0.1098, 0.1098, 0.0392],
          [0.0902, 0.1098, 0.1137,  ..., 0.1098, 0.1098, 0.0392],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0078, 0.0078, 0.0078],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0.0941, 0.1098, 0.1098,  ..., 0.1059, 0.1137, 0.0392],
          [0.0941, 0.1098, 0.1098,  ..., 0.1098, 0.1098, 0.0392],
          [0.0902, 0.1098, 0.1137,  ...,

## Model Setup

In [9]:
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim
from torchvision.transforms import ToTensor
from PIL import Image

In [10]:

class CustomNeuralNetwork(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super().__init__()

        self.LeNet = nn.Sequential(     
            # convolutional layers            
            nn.Sequential(                                            # FIRST LAYER: (INPUT LAYER)
              nn.Conv2d(3, 6, kernel_size=3, stride=1, padding=0),    # CONVOLUTION 
              nn.BatchNorm2d(6),
              nn.Softmax(dim=1),
              nn.Dropout(p=dropout_rate),
              nn.MaxPool2d(kernel_size=2, stride=2)),                # POOLING
            nn.Sequential(                                            # SECOND LAYER: HIDDEN LAYER 1
              nn.Conv2d(6, 16, kernel_size=3, stride=1, padding=0),   # CONVOLUTION 
              nn.BatchNorm2d(16),
              nn.Softmax(dim=1),
              nn.Dropout(p=dropout_rate),
              nn.MaxPool2d(kernel_size=2, stride=2)),                # POOLING
            # fully connected layers
            nn.Flatten(),
            # output layer
            nn.Linear(16 * 54 * 54, 2)                                # OUTPUT LAYER
        )

    def forward(self, x):
        out = self.LeNet(x)
        return out




## Softmax + Negative Log Likelihood Loss

In [11]:

class CustomNeuralNetwork(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super().__init__()

        self.LeNet = nn.Sequential(     
            # convolutional layers            
            nn.Sequential(                                            # FIRST LAYER: (INPUT LAYER)
              nn.Conv2d(3, 6, kernel_size=3, stride=1, padding=0),    # CONVOLUTION 
              nn.BatchNorm2d(6),
              nn.Softmax(dim=1),
              nn.Dropout(p=dropout_rate),
              nn.MaxPool2d(kernel_size=2, stride=2)),                # POOLING
            nn.Sequential(                                            # SECOND LAYER: HIDDEN LAYER 1
              nn.Conv2d(6, 16, kernel_size=3, stride=1, padding=0),   # CONVOLUTION 
              nn.BatchNorm2d(16),
              nn.Softmax(dim=1),
              nn.Dropout(p=dropout_rate),
              nn.MaxPool2d(kernel_size=2, stride=2)),                # POOLING
            # fully connected layers
            nn.Flatten(),
            # output layer
            nn.Linear(16 * 54 * 54, 2)                                # OUTPUT LAYER
        )

    def forward(self, x):
        out = self.LeNet(x)
        return F.log_softmax(out, dim=1)



In [12]:
model_nll = CustomNeuralNetwork()
criterion = nn.NLLLoss()
optimizer = optim.SGD(model_nll.parameters(), lr=0.001, momentum=0.9)

In [13]:
EPOCHS = 20

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

for epoch in range(EPOCHS):  # loop over the dataset multiple times

    # TRAIN
    # Make sure gradient tracking is on, and do a pass over the data
    model_nll.train()
    running_loss = 0.0
    running_matched = 0
    for i, data in enumerate(train_dataloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data           # NOTE: depending on how you implemented your dataset class's __getitem__ it could be labels, inputs

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model_nll(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # keep track of the loss
        running_loss += loss.item()

        # CALCULATE ACCURACY METRIC
        _, preds = torch.max(outputs, 1)  # Find out the predicted class with the highest prob
        running_matched += torch.sum(preds == labels.data) # caculate the number of matched labels

    avg_train_loss = running_loss / (i + 1)     # i + 1 gives us the total number of batches in train dataloader
    avg_train_acc = running_matched.double() / len(train_dataloader.dataset)
    train_losses.append(avg_train_loss)
    train_accuracies.append(avg_train_acc)

    # VALIDATE
    # In the validation part, we don't want to keep track of the gradients 
    model_nll.eval()
    running_val_loss = 0.0
    running_val_matched = 0
    with torch.no_grad():
        for i, data in enumerate(val_dataloader):
            inputs, labels = data
            outputs = model_nll(inputs)
            loss = criterion(outputs, labels)

            # keep track of the loss
            running_val_loss += loss.item()

            # CALCULATE ACCURACY METRIC
            _, preds = torch.max(outputs, 1)
            running_val_matched += torch.sum(preds == labels.data)

    avg_val_loss = running_val_loss / (i + 1)
    avg_val_acc = running_val_matched.double() / len(val_dataloader.dataset)
    val_losses.append(avg_val_loss)
    val_accuracies.append(avg_val_acc)

    # Print epoch summary
    print(f"Epoch {epoch+1}/{EPOCHS} | "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f} | "
          f"Val Loss: {avg_val_loss:.4f}, Val Acc: {avg_val_acc:.4f}")



Epoch 1/20 | Train Loss: 0.3262, Train Acc: 0.8822 | Val Loss: 0.6457, Val Acc: 0.5625
Epoch 2/20 | Train Loss: 0.1758, Train Acc: 0.9410 | Val Loss: 0.8021, Val Acc: 0.5000
Epoch 3/20 | Train Loss: 0.1386, Train Acc: 0.9545 | Val Loss: 0.7471, Val Acc: 0.5000
Epoch 4/20 | Train Loss: 0.1186, Train Acc: 0.9602 | Val Loss: 0.8547, Val Acc: 0.5000
Epoch 5/20 | Train Loss: 0.1085, Train Acc: 0.9657 | Val Loss: 0.8934, Val Acc: 0.5000
Epoch 6/20 | Train Loss: 0.0995, Train Acc: 0.9675 | Val Loss: 0.8840, Val Acc: 0.5000
Epoch 7/20 | Train Loss: 0.0924, Train Acc: 0.9700 | Val Loss: 0.9567, Val Acc: 0.5000
Epoch 8/20 | Train Loss: 0.0861, Train Acc: 0.9727 | Val Loss: 0.9420, Val Acc: 0.5000
Epoch 9/20 | Train Loss: 0.0798, Train Acc: 0.9728 | Val Loss: 0.8896, Val Acc: 0.5000
Epoch 10/20 | Train Loss: 0.0748, Train Acc: 0.9751 | Val Loss: 1.0771, Val Acc: 0.5000
Epoch 11/20 | Train Loss: 0.0736, Train Acc: 0.9762 | Val Loss: 0.9686, Val Acc: 0.5000
Epoch 12/20 | Train Loss: 0.0715, Train A

In [14]:
torch.save(model_nll.state_dict(), './softmaxmodel_nll.PT')
#To load do this:
#model = torch.load('./softmaxmodel_nll.PT')


In [15]:
train_accuracies

[tensor(0.8822, dtype=torch.float64),
 tensor(0.9410, dtype=torch.float64),
 tensor(0.9545, dtype=torch.float64),
 tensor(0.9602, dtype=torch.float64),
 tensor(0.9657, dtype=torch.float64),
 tensor(0.9675, dtype=torch.float64),
 tensor(0.9700, dtype=torch.float64),
 tensor(0.9727, dtype=torch.float64),
 tensor(0.9728, dtype=torch.float64),
 tensor(0.9751, dtype=torch.float64),
 tensor(0.9762, dtype=torch.float64),
 tensor(0.9751, dtype=torch.float64),
 tensor(0.9786, dtype=torch.float64),
 tensor(0.9823, dtype=torch.float64),
 tensor(0.9796, dtype=torch.float64),
 tensor(0.9805, dtype=torch.float64),
 tensor(0.9827, dtype=torch.float64),
 tensor(0.9847, dtype=torch.float64),
 tensor(0.9847, dtype=torch.float64),
 tensor(0.9839, dtype=torch.float64)]

In [16]:
train_losses

[0.32616804684362105,
 0.1758161514036117,
 0.138580015229602,
 0.11861246169334458,
 0.10846087183322638,
 0.0995263522490859,
 0.09239630869800045,
 0.08612944155691131,
 0.07978518612142052,
 0.07481737751814146,
 0.07360889199340055,
 0.07151141511877218,
 0.06544885727306528,
 0.06079434483282028,
 0.06095229207928623,
 0.05734307994885791,
 0.05401256589609529,
 0.05071916461231247,
 0.05251383390127411,
 0.050730077947880474]

In [17]:
val_accuracies

[tensor(0.5625, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64)]

In [18]:
val_losses

[0.6457382440567017,
 0.8020821809768677,
 0.7471135258674622,
 0.8546653985977173,
 0.8934195041656494,
 0.8840035200119019,
 0.9567467570304871,
 0.9419949054718018,
 0.8895538449287415,
 1.0771489143371582,
 0.9686247706413269,
 1.1260128021240234,
 1.133291482925415,
 0.9450309872627258,
 1.3620797395706177,
 0.9776105880737305,
 1.1788190603256226,
 1.202968955039978,
 1.184328317642212,
 1.170414686203003]

In [19]:
model_nll.eval()
running_test_loss = 0.0
running_test_matched = 0
with torch.no_grad():
    for i, data in enumerate(test_dataloader):
        inputs, labels = data
        outputs = model_nll(inputs)
        loss = criterion(outputs, labels)

        # keep track of the loss
        running_test_loss += loss.item()

        # CALCULATE ACCURACY METRIC
        _, preds = torch.max(outputs, 1)
        running_test_matched += torch.sum(preds == labels.data)

avg_test_loss = running_test_loss
avg_test_acc = running_test_matched.double() / len(test_dataloader.dataset)

# Print epoch summary
print("Test Loss: {}".format(avg_test_loss))
print("Test Acc: {}".format(avg_test_acc))

Test Loss: 1.0381602048873901
Test Acc: 0.5


## Softmax + Cross Entropy

In [12]:
model_1 = CustomNeuralNetwork()

In [30]:
# Define a Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_1.parameters(), lr=0.001, momentum=0.9)

In [32]:
EPOCHS = 20

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

for epoch in range(EPOCHS):  # loop over the dataset multiple times

    # TRAIN
    # Make sure gradient tracking is on, and do a pass over the data
    model_1.train()
    running_loss = 0.0
    running_matched = 0
    for i, data in enumerate(train_dataloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data           # NOTE: depending on how you implemented your dataset class's __getitem__ it could be labels, inputs

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model_1(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # keep track of the loss
        running_loss += loss.item()

        # CALCULATE ACCURACY METRIC
        _, preds = torch.max(outputs, 1)  # Find out the predicted class with the highest prob
        running_matched += torch.sum(preds == labels.data) # caculate the number of matched labels

    avg_train_loss = running_loss / (i + 1)     # i + 1 gives us the total number of batches in train dataloader
    avg_train_acc = running_matched.double() / len(train_dataloader.dataset)
    train_losses.append(avg_train_loss)
    train_accuracies.append(avg_train_acc)

    # VALIDATE
    # In the validation part, we don't want to keep track of the gradients 
    model_1.eval()
    running_val_loss = 0.0
    running_val_matched = 0
    with torch.no_grad():
        for i, data in enumerate(val_dataloader):
            inputs, labels = data
            outputs = model_1(inputs)
            loss = criterion(outputs, labels)

            # keep track of the loss
            running_val_loss += loss.item()

            # CALCULATE ACCURACY METRIC
            _, preds = torch.max(outputs, 1)
            running_val_matched += torch.sum(preds == labels.data)

    avg_val_loss = running_val_loss / (i + 1)
    avg_val_acc = running_val_matched.double() / len(val_dataloader.dataset)
    val_losses.append(avg_val_loss)
    val_accuracies.append(avg_val_acc)

    # Print epoch summary
    print(f"Epoch {epoch+1}/{EPOCHS} | "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f} | "
          f"Val Loss: {avg_val_loss:.4f}, Val Acc: {avg_val_acc:.4f}")


Epoch 1/20 | Train Loss: 0.1017, Train Acc: 0.9663 | Val Loss: 0.5162, Val Acc: 0.8125
Epoch 2/20 | Train Loss: 0.0907, Train Acc: 0.9713 | Val Loss: 0.6804, Val Acc: 0.5000
Epoch 3/20 | Train Loss: 0.0835, Train Acc: 0.9720 | Val Loss: 0.5998, Val Acc: 0.5000
Epoch 4/20 | Train Loss: 0.0762, Train Acc: 0.9730 | Val Loss: 0.5667, Val Acc: 0.6250
Epoch 5/20 | Train Loss: 0.0716, Train Acc: 0.9773 | Val Loss: 0.4982, Val Acc: 0.7500
Epoch 6/20 | Train Loss: 0.0704, Train Acc: 0.9768 | Val Loss: 0.4728, Val Acc: 0.7500
Epoch 7/20 | Train Loss: 0.0661, Train Acc: 0.9782 | Val Loss: 0.5531, Val Acc: 0.6250
Epoch 8/20 | Train Loss: 0.0604, Train Acc: 0.9804 | Val Loss: 0.4017, Val Acc: 0.8750
Epoch 9/20 | Train Loss: 0.0571, Train Acc: 0.9816 | Val Loss: 0.4853, Val Acc: 0.8125
Epoch 10/20 | Train Loss: 0.0528, Train Acc: 0.9844 | Val Loss: 0.6948, Val Acc: 0.5000
Epoch 11/20 | Train Loss: 0.0513, Train Acc: 0.9844 | Val Loss: 0.7799, Val Acc: 0.5000
Epoch 12/20 | Train Loss: 0.0505, Train A

In [35]:
train_accuracies

[tensor(0.9663, dtype=torch.float64),
 tensor(0.9713, dtype=torch.float64),
 tensor(0.9720, dtype=torch.float64),
 tensor(0.9730, dtype=torch.float64),
 tensor(0.9773, dtype=torch.float64),
 tensor(0.9768, dtype=torch.float64),
 tensor(0.9782, dtype=torch.float64),
 tensor(0.9804, dtype=torch.float64),
 tensor(0.9816, dtype=torch.float64),
 tensor(0.9844, dtype=torch.float64),
 tensor(0.9844, dtype=torch.float64),
 tensor(0.9842, dtype=torch.float64),
 tensor(0.9838, dtype=torch.float64),
 tensor(0.9849, dtype=torch.float64),
 tensor(0.9877, dtype=torch.float64),
 tensor(0.9877, dtype=torch.float64),
 tensor(0.9896, dtype=torch.float64),
 tensor(0.9889, dtype=torch.float64),
 tensor(0.9906, dtype=torch.float64),
 tensor(0.9886, dtype=torch.float64)]

In [39]:
train_losses

[0.10173026018685871,
 0.09072341884095822,
 0.08354679867625237,
 0.07615594978955004,
 0.07161806333028982,
 0.07035121841416243,
 0.06614582539506016,
 0.06039045020307024,
 0.05711717502544484,
 0.052785396883865035,
 0.051331473702204325,
 0.05054859138063846,
 0.04812943094199704,
 0.044066273877697605,
 0.04058976383549311,
 0.040024005011805606,
 0.0375443015457882,
 0.038703201511394114,
 0.034343408585916606,
 0.037115964159790064]

In [37]:
val_accuracies

[tensor(0.8125, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.6250, dtype=torch.float64),
 tensor(0.7500, dtype=torch.float64),
 tensor(0.7500, dtype=torch.float64),
 tensor(0.6250, dtype=torch.float64),
 tensor(0.8750, dtype=torch.float64),
 tensor(0.8125, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.5000, dtype=torch.float64),
 tensor(0.9375, dtype=torch.float64),
 tensor(0.8750, dtype=torch.float64),
 tensor(0.9375, dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(0.8125, dtype=torch.float64),
 tensor(0.8125, dtype=torch.float64),
 tensor(0.8750, dtype=torch.float64),
 tensor(0.9375, dtype=torch.float64),
 tensor(0.9375, dtype=torch.float64)]

In [43]:
val_losses

[0.5162,
 0.6804,
 0.5998,
 0.5667,
 0.4982,
 0.4728,
 0.5531,
 0.4017,
 0.4853,
 0.6948,
 0.7799,
 0.3926,
 0.3511,
 0.3279,
 0.3167,
 0.3746,
 0.4695,
 0.3311,
 0.3432,
 0.3215]

In [50]:
torch.save(model_1.state_dict(), './softmaxmodel_ce.PT')

In [3]:
#To load do this:
model = torch.load('./softmaxmodel_ce.PT')

In [4]:
model

OrderedDict([('LeNet.0.0.weight',
              tensor([[[[-0.1343, -0.0068,  0.0198],
                        [-0.1361,  0.0117,  0.1259],
                        [ 0.0837, -0.1172, -0.1514]],
              
                       [[-0.1624,  0.1657, -0.0845],
                        [-0.1871, -0.1246, -0.0627],
                        [-0.0899, -0.1158, -0.1532]],
              
                       [[-0.1476, -0.1400,  0.1144],
                        [ 0.0564, -0.0901, -0.0535],
                        [-0.1150,  0.1545, -0.0719]]],
              
              
                      [[[-0.0372,  0.0714, -0.1404],
                        [ 0.0102,  0.0635,  0.1555],
                        [ 0.0948, -0.1285, -0.0103]],
              
                       [[-0.0273,  0.0849,  0.0087],
                        [ 0.1437,  0.0448,  0.0980],
                        [-0.1116, -0.0326, -0.0358]],
              
                       [[ 0.1237, -0.0710,  0.0915],
                      

In [59]:
#TEST

model_1.eval()
running_test_loss = 0.0
running_test_matched = 0
with torch.no_grad():
    for i, data in enumerate(test_dataloader):
        inputs, labels = data
        outputs = model_1(inputs)
        loss = criterion(outputs, labels)

        # keep track of the loss
        running_test_loss += loss.item()

        # CALCULATE ACCURACY METRIC
        _, preds = torch.max(outputs, 1)
        running_test_matched += torch.sum(preds == labels.data)

avg_test_loss = running_test_loss
avg_test_acc = running_test_matched.double() / len(test_dataloader.dataset)

# Print epoch summary
print("Test Loss: {}".format(avg_test_loss))
print("Test Acc: {}".format(avg_test_acc))

Test Loss: 0.42320847511291504
Test Acc: 0.875
